In [ ]:
import sys
import numpy as np
import random
import os
import shutil
import pickle
import time
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
random.seed(42)
np.random.seed(42)

In [ ]:
def sort_transition_prob_dict(transition_probs):
    sorted_transition_probs = {}
    for source in transition_probs:
        sorted_targets = [k for k, v in sorted(transition_probs[source].items(), key=lambda item: item[1], reverse=True)]
        sorted_transition_probs[source] = sorted_targets
    return sorted_transition_probs

In [ ]:
def build_transition_prob_dict(paths, markov_order=1, min_session_len=2):
    transition_probs = {}; num_sessions = 0
    for lnum, line in enumerate(tqdm(paths)):
        session = line.strip().split(" ")
        N = len(session)
        if N<min_session_len:
            continue
        for idx in range(N-markov_order):
            src_ids = [int(session[idx+j]) for j in range(markov_order)]
            trgt_id = int(session[idx+markov_order])
            src_id = tuple(src_ids) if markov_order>=2 else src_ids[0]

            if src_id in transition_probs:
                if trgt_id in transition_probs[src_id]:
                    transition_probs[src_id][trgt_id] += 1
                else:
                    transition_probs[src_id][trgt_id] = 1
            else:
                transition_probs[src_id] = {trgt_id: 1}
        num_sessions+=1
    return transition_probs, num_sessions

In [ ]:
def prepare_transition_probs(paths, markov_order=1, min_session_len=2):
    transition_probs, num_sessions = build_transition_prob_dict(paths, markov_order, min_session_len)
    sorted_transition_probs = sort_transition_prob_dict(transition_probs)
    return sorted_transition_probs, num_sessions

In [ ]:
def save_model(model, fname):
    fmodel_out = open(fname,"wb")
    pickle.dump(model, fmodel_out)

In [ ]:
def load_model(fname):
    model = pickle.load(open(fname,'rb'))
    return model

In [ ]:
def prepare_queries(paths, N_max = -1, markov_order=1, min_session_len=2):
    '''
    get pairs of source-target articles from a file containing reading sessions.
    from a file containing sequences of pageview.
    select all pairs of consecutive pageivews.
    returns a list of tuples [(src,trg)], where src, trg are of type str.

    get at most pairs from N_max sessions (default is -1 == all).
    '''
    queries = []
    count=0

    for line in paths:
        session = line.strip().split(" ")

        N = len(session)
        if N<min_session_len:
            continue

        for idx in range(N-markov_order):
            src_ids = [int(session[idx+j]) for j in range(markov_order)]
            trgt_id = int(session[idx+markov_order])
            src_id = tuple(src_ids) if markov_order>=2 else src_ids[0]

            queries.append((N, (src_id,trgt_id)))
        count+=1
        if count == N_max:
            break
    return queries, count

In [ ]:
def queriesRanks(queries, transition_probs, markov_order=1):
    '''
    from a list of pairs (src,target)
    - get the nearest neighbors of src via the first-order-markov model inferred from the training reader sessions
    - check rank of trg among nearest neighbors
    '''
    lengthwise_rank_list = {}
    rank_list = []
    missing_sources = 0; missing_targets = 0
    for sess_len, (pid_src,pid_trg) in queries:
        try:
            if type(pid_src) == tuple:
                if markov_order == 1:
                    sorted_targets = transition_probs[pid_src[-1]]
                else:
                    sorted_targets = transition_probs[pid_src]
            else:
                sorted_targets = transition_probs[pid_src]
            rank = sorted_targets.index(pid_trg)+1
        except KeyError:
            rank = 1e6
            missing_sources+=1
        except ValueError:
            rank = 1e6
            missing_targets +=1

        if sess_len not in lengthwise_rank_list:
            lengthwise_rank_list[sess_len] = [rank]
        else:
            lengthwise_rank_list[sess_len].append(rank)

        rank_list.append(rank)
    print(f'#Missing-sources = {missing_sources}; #Missing-targets = {missing_targets}')
    return np.array(rank_list), lengthwise_rank_list

In [ ]:
def check_query_skip_status(query, trgt, model_list):
    skip_query = False
    for model in model_list:
        if query not in model:
            skip_query = True
            break
        else:
            if trgt not in model[query]:
                skip_query = True
                break
    return skip_query

In [ ]:
def queriesRanksNoMissing(queries, model_list, model, markov_order=1):
    '''
    from a list of pairs (src,target)
    - get the nearest neighbors of src via the first-order-markov model inferred from the training reader sessions
    - check rank of trg among nearest neighbors
    '''
    lengthwise_rank_list = {}
    rank_list = []
    skipped_count = 0; missing_sources = 0; missing_targets = 0
    for sess_len, (pid_src,pid_trg) in queries:
        if type(pid_src) == tuple:
            if markov_order == 1:
                query = pid_src[-1]
            else:
                query = pid_src
        else:
            query = pid_src
        skip_query = check_query_skip_status(query, pid_trg, model_list)
        if skip_query:
            skipped_count += 1
        else:
            try:
                sorted_targets = model[query]
                rank = sorted_targets.index(pid_trg)+1
            except KeyError:
                rank = 1e6
                missing_sources+=1
            except ValueError:
                rank = 1e6
                missing_targets +=1
            if sess_len not in lengthwise_rank_list:
                lengthwise_rank_list[sess_len] = [rank]
            else:
                lengthwise_rank_list[sess_len].append(rank)
            rank_list.append(rank)

    print(f'#Skipped-queries = {skipped_count}; #Missing-sources = {missing_sources}; #Missing-targets = {missing_targets}')
    return np.array(rank_list), lengthwise_rank_list

In [ ]:
def query_pruned_and_write_results(path_type, minlen, num_sessions, queries, model_list, model, model_order, query_order, results_fname, lenresults_fname):
    if not os.path.isfile(results_fname):
        fout = open(results_fname, 'w'); fout_len = open(lenresults_fname, 'w')
        fout.write("PathType\tMinSessionLength\tMarkovOrder\tQueryOrder\t#Sessions\t#Queries\t#Queries_Predicted\tRecall@1\tRecall@5\tRecall@10\tRecall@50\tRecall@100\tMRR\n")
        fout_len.write("PathType\tMinSessionLength\tMarkovOrder\tQueryOrder\t#Sessions\tQuerySessionLength\t#Queries\t#Queries_Predicted\tRecall@1\tRecall@5\tRecall@10\tRecall@50\tRecall@100\tMRR\n")
    else:
        fout = open(results_fname, 'a'); fout_len = open(lenresults_fname, 'a')

    ranks_queries, lengthwise_ranks_queries = queriesRanksNoMissing(queries, model_list, model, model_order)
    nqueries, recall1, recall5, recall10, recall50, recall100, mrr = metrics(ranks_queries)
    fout.write(f'{path_type}\t{minlen}\t{model_order}\t{query_order}\t{num_sessions}\t{len(queries)}\t{nqueries}\t{recall1}\t{recall5}\t{recall10}\t{recall50}\t{recall100}\t{mrr}\n')
    for session_length in sorted(lengthwise_ranks_queries):
        rank_list = lengthwise_ranks_queries[session_length]
        nqueries, recall1, recall5, recall10, recall50, recall100, mrr = metrics(np.array(rank_list))
        fout_len.write(f'{path_type}\t{minlen}\t{model_order}\t{query_order}\t{num_sessions}\t{session_length}\t{len(queries)}\t{nqueries}\t{recall1}\t{recall5}\t{recall10}\t{recall50}\t{recall100}\t{mrr}\n')
    fout.close()
    fout_len.close()

In [ ]:
def query_and_write_results(path_type, minlen, num_sessions, queries, model, model_order, query_order, results_fname, lenresults_fname):
    if not os.path.isfile(results_fname):
        fout = open(results_fname, 'w'); fout_len = open(lenresults_fname, 'w')
        fout.write("PathType\tMinSessionLength\tMarkovOrder\tQueryOrder\t#Sessions\t#Queries\t#Queries_Predicted\tRecall@1\tRecall@5\tRecall@10\tRecall@50\tRecall@100\tMRR\n")
        fout_len.write("PathType\tMinSessionLength\tMarkovOrder\tQueryOrder\t#Sessions\tQuerySessionLength\t#Queries\t#Queries_Predicted\tRecall@1\tRecall@5\tRecall@10\tRecall@50\tRecall@100\tMRR\n")
    else:
        fout = open(results_fname, 'a'); fout_len = open(lenresults_fname, 'a')

    ranks_queries, lengthwise_ranks_queries = queriesRanks(queries, model, markov_order=model_order)
    nqueries, recall1, recall5, recall10, recall50, recall100, mrr = metrics(ranks_queries)
    fout.write(f'{path_type}\t{minlen}\t{model_order}\t{query_order}\t{num_sessions}\t{len(queries)}\t{nqueries}\t{recall1}\t{recall5}\t{recall10}\t{recall50}\t{recall100}\t{mrr}\n')
    for session_length in sorted(lengthwise_ranks_queries):
        rank_list = lengthwise_ranks_queries[session_length]
        nqueries, recall1, recall5, recall10, recall50, recall100, mrr = metrics(np.array(rank_list))
        fout_len.write(f'{path_type}\t{minlen}\t{model_order}\t{query_order}\t{num_sessions}\t{session_length}\t{len(queries)}\t{nqueries}\t{recall1}\t{recall5}\t{recall10}\t{recall50}\t{recall100}\t{mrr}\n')
    fout.close()
    fout_len.close()

In [ ]:
def metrics(mrr_list):
    mrr = np.mean(1/mrr_list)
    recall1 = np.where((mrr_list <= 1) & (mrr_list != 1e6))[0].shape[0]/mrr_list.shape[0]
    recall5 = np.where((mrr_list <= 5) & (mrr_list != 1e6))[0].shape[0]/mrr_list.shape[0]
    recall10 = np.where((mrr_list <= 10) & (mrr_list != 1e6))[0].shape[0]/mrr_list.shape[0]
    recall50 = np.where((mrr_list <= 50) & (mrr_list != 1e6))[0].shape[0]/mrr_list.shape[0]
    recall100 = np.where((mrr_list <= 100) & (mrr_list != 1e6))[0].shape[0]/mrr_list.shape[0]
    return mrr_list.shape[0], recall1, recall5, recall10, recall50, recall100, mrr

In [ ]:
def read_paths_random_shuffle(paths_fname):
    paths_full = []
    for line in tqdm(open(paths_fname)):
        line = line.strip().split(",")
        paths_full.append(line[1])
    tsplit = time.time()
    random.shuffle(paths_full)
    num_train = int(0.8 * len(paths_full))
    num_val = int(0.1* len(paths_full))
    num_test = len(paths_full) - num_train - num_val
    print(num_train, num_val, num_test)
    paths_train = paths_full[0:num_train]; paths_val = paths_full[num_train:num_train+num_val]; paths_test = paths_full[num_train+num_val:]
    print(f'Splitting using random shuffle took {time.time()-tsplit} seconds')
    return paths_train, paths_val, paths_test

In [ ]:
def train_val_test_split(paths_full):
    paths_train, paths_test = train_test_split(paths_full, test_size=0.2, random_state=42)
    paths_val, paths_test = train_test_split(paths_test, test_size=0.5, random_state=42)
    return paths_train, paths_val, paths_test

In [ ]:
def read_paths(paths_fname):
    paths_full = []
    for lnum, line in enumerate(tqdm(open(paths_fname))):
        if lnum == 0:
            continue
        line = line.strip().split(",")
        paths_full.append(line[1])
    return paths_full

In [ ]:
root_dir = os.path.abspath(os.path.join(os.getcwd(),os.pardir))
PATH_IN = root_dir
langlist = ['en', 'ru', 'ja', 'de', 'fr', 'it', 'pl', 'fa']
path_types = ['real_nav', 'gen_clickstream_private', 'gen_clickstream_public', 'gen_graph']

In [ ]:
for lang in langlist:
    print(f'{lang}wiki')
    out_dir = os.path.join(PATH_IN, 'downstream_tasks', 'next_article_results', lang)
    if os.path.isdir(out_dir):
        shutil.rmtree(out_dir)
    os.makedirs(out_dir)
    model = {}
    for path_type in path_types:
        model[path_type] = {}
        print(f'Paths: {path_type}')
        paths_fname = os.path.join(PATH_IN, 'data', 'navigation_paths', lang, f'paths_{path_type}.csv')
        
        tread = time.time()
        paths_full = read_paths(paths_fname)
        if path_type == 'real_nav':
            paths_train, paths_val, paths_test = train_val_test_split(paths_full)
        else:
            paths_train, _, _ = train_val_test_split(paths_full)
        print(len(paths_train), len(paths_val), len(paths_test))
        print(f'Reading {paths_fname} took {time.time()-tread} seconds')

        queries_dict = {}
        for minlen in [2,3]:
            model[path_type][minlen] = {}; queries_dict[minlen] = {}
            for order in [1,2]:
                if order == minlen:
                    continue
                model_out_fname = os.path.join(PATH_IN, 'data', 'models', lang, f"markov_model_{path_type}_minlen{minlen}_order{order}.pickle")
                results_fname = os.path.join(out_dir, f"{lang}wiki_nextarticle_prediction_minlen{minlen}_modelorder{order}_queryorder{order}.tsv")
                lenresults_fname = os.path.join(out_dir, f"{lang}wiki_nextarticle_prediction_minlen{minlen}_modelorder{order}_queryorder{order}_lengthwise.tsv")

                tmodel = time.time()
                transition_probs, num_sessions = prepare_transition_probs(paths_train, markov_order=order, min_session_len=minlen)
                print(f'Building ProbMap for Markov order = {order} with sessions of length at least {minlen} took {time.time()-tmodel} seconds')
                model[path_type][minlen][order] = (transition_probs, num_sessions)
                save_model(transition_probs, model_out_fname)

                tqueryprep = time.time()
                queries, num_sessions = prepare_queries(paths_test, -1, markov_order=order, min_session_len=minlen)
                queries_dict[minlen][order] = (queries, num_sessions)
                print(f'Prepared {len(queries)} queries from {num_sessions} sessions in {time.time()-tqueryprep} seconds')
                
                tquery = time.time()
                query_and_write_results(path_type, minlen, num_sessions, queries, transition_probs, order, order, results_fname, lenresults_fname)
                print(f'Querying model with markov_order={order} for {len(queries)} queries with min session length {minlen} took {time.time() - tquery} seconds')

        results_fname = os.path.join(out_dir, f"{lang}wiki_nextarticle_prediction_minlen{minlen}_modelorder1_queryorder2.tsv")
        lenresults_fname = os.path.join(out_dir, f"{lang}wiki_nextarticle_prediction_minlen{minlen}_modelorder1_queryorder2_lengthwise.tsv")
        tquery = time.time()
        query_and_write_results(path_type, 3, queries_dict[3][2][1], queries_dict[3][2][0], model[path_type][3][1][0], 1, 2, results_fname, lenresults_fname)
        print(f'Querying model with markov_order=1 for {len(queries_dict[3][2][0])} second-order queries with min session length 3 took {time.time() - tquery} seconds')

    print(f'{lang}wiki pruned queries')
    for minlen, model_order, query_order in [(2,1,1), (3,1,1), (3,1,2), (3,2,2)]:
        tload_start = time.time()
        model_list = []
        for path_type in path_types:
            if path_type != 'gen_graph':
                model_list.append(model[path_type][minlen][model_order][0])
        tload_end = time.time()

        for path_type in path_types:
            results_fname = os.path.join(out_dir, f"{lang}wiki_nextarticle_prediction_prunedqueries_minlen{minlen}_modelorder{model_order}_queryorder{query_order}.tsv")
            lenresults_fname = os.path.join(out_dir, f"{lang}wiki_nextarticle_prediction_prunedqueries_minlen{minlen}_modelorder{model_order}_queryorder{query_order}_lengthwise.tsv")
            tquery = time.time()
            query_pruned_and_write_results(path_type, minlen, queries_dict[minlen][query_order][1], queries_dict[minlen][query_order][0], model_list, model[path_type][minlen][model_order][0], model_order, query_order, results_fname, lenresults_fname)
            print(f'Querying model with markov_order={model_order} for {len(queries_dict[minlen][query_order][0])} queries of query_order={query_order} with min session length {minlen} took {(tload_end - tload_start) + time.time() - tquery} seconds')